<a href="https://colab.research.google.com/github/saranoor/chatGPT/blob/master/qs_ans_using_prestored_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken

COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"

In [ ]:
from openai.embeddings_utils import get_embedding
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [ ]:
API_KEY = '' # Add your API key here
openai.api_key = API_KEY

In [ ]:
import csv

In [ ]:
def parse_float_list(string):
    # Remove brackets and split by comma
    values = string.strip('[]').split(',')
    # Convert each value to a float and return as a list
    if len(values) == 1 and values[0] == '':
      return np.nan
    return [float(value) for value in values]

In [ ]:
df_test=pd.read_csv('qs_ans_embeddings_updated.csv', converters={'embeddings': parse_float_list})

In [ ]:
df_test

,Unnamed: 0,title,heading,content,combined,n_tokens,embeddings
0,0,10Pearls Policies and Procedures,OUR MISSION,TenPearls (Private) Limited is founded on the ...,Title: 10Pearls Policies and Procedures; Headi...,109,"[-0.003407263197004795, -0.022962724789977074,..."
1,1,Definitions in 10Pearls Policies and Procedures,DEFINITIONS,In this Manual the following words and express...,Title: Definitions in 10Pearls Policies and Pr...,41,"[-0.022202739492058754, -0.005969981197267771,..."
2,2,Definitions in 10Pearls Policies and Procedures,Company,“Company” shall mean TenPearls (Private) Limited;,Title: Definitions in 10Pearls Policies and Pr...,31,"[-0.011864437721669674, -0.004838661756366491,..."
3,3,Definitions in 10Pearls Policies and Procedures,PULSE,“PULSE” shall mean the Company online portal a...,Title: Definitions in 10Pearls Policies and Pr...,37,"[-0.03371162340044975, 0.0023054599296301603, ..."
4,4,Definitions in 10Pearls Policies and Procedures,Employee,“Employee(s)” shall mean all employees of the ...,Title: Definitions in 10Pearls Policies and Pr...,54,"[-0.026462620124220848, 0.0019038093741983175,..."
...,...,...,...,...,...,...,...
139,140,APPENDIX J in 10Pearls Policies and Procedures,CEILING AMOUNT,Level\nManagers and Asst. Managers/Architects\...,Title: APPENDIX J in 10Pearls Policies and Pro...,88,"[0.0009748903103172779, -0.0001310141815338283..."
140,141,APPENDIX K in 10Pearls Policies and Procedures,PURPOSE,The Company has designed LTIP to reward its el...,Title: APPENDIX K in 10Pearls Policies and Pro...,120,"[-0.016293466091156006, -0.007967986166477203,..."
141,142,APPENDIX K in 10Pearls Policies and Procedures,PARTICIPANTS FOR THE LTIP UNITS,• The officers and other senior executive Conf...,Title: APPENDIX K in 10Pearls Policies and Pro...,528,"[-0.0195445753633976, -0.006254264153540134, 0..."
142,143,APPENDIX K in 10Pearls Policies and Procedures,MINIMUM HOLDING PERIOD AND ACTUAL HOLDING PERIOD,The Minimum Holding Period for LTIP Units issu...,Title: APPENDIX K in 10Pearls Policies and Pro...,333,"[-0.019007259979844093, 0.0036615715362131596,..."


In [ ]:
df_test.drop(['Unnamed: 0'],axis=1, inplace=True)

In [ ]:
def load_embeddings(fname: str):
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    # df=pd.read_excel(fname)
    return {
            (r.title, r.heading): r.embeddings for _, r in df_test.iterrows()
    }

In [ ]:
test_doc_embeddings=load_embeddings('.csv')

In [ ]:
list(test_doc_embeddings.items())[121:122]

[(('APPENDIX E LOAN POLICY in 10Pearls Policies and Procedures',
   'POLICY OF LOAN AMOUNT'),
  [-0.006558274384588003,
   0.0033833181951195,
   0.027749698609113693,
   -0.04183632507920265,
   -0.00733023788779974,
   0.0022339122369885445,
   -0.006708567962050438,
   0.008573577739298344,
   -0.023145243525505066,
   -0.0116750942543149,
   0.011955187655985355,
   0.0009094481938518584,
   -0.021191423758864403,
   0.01394999586045742,
   -0.03672633692622185,
   0.012378742918372154,
   0.0040681795217096806,
   -0.02156032621860504,
   -0.00844377838075161,
   -0.002507173689082265,
   -0.02483946457505226,
   0.0014961063861846924,
   -0.020262334495782852,
   0.00832081027328968,
   0.005212462041527033,
   -0.010582049377262592,
   0.02522202953696251,
   -0.020098378881812096,
   0.005974178202450275,
   -0.003938380628824234,
   0.030058758333325386,
   0.00539008155465126,
   -0.00990572664886713,
   0.002701872494071722,
   -0.01616341434419155,
   0.004204810131341219,


In [ ]:
# An example embedding:
example_entry = list(test_doc_embeddings.items())[120]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

('APPENDIX E LOAN POLICY in 10Pearls Policies and Procedures', 'GENERAL') : [-0.00859899539500475, -0.007440510205924511, 0.01860472373664379, -0.04101589694619179, 0.018521973863244057]... (1536 entries)


In [ ]:
def vector_similarity(x, y):
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    # print(type(x), type(y))
    # x = np.array(x)
    # y = np.array(y)

    x = np.array(x,dtype="float64")
    y = np.array(y,dtype="float64")
    if np.isnan(x).any():
      return 0.0
    elif np.isnan(y).any():
      return 0.0
    return np.dot(x, y)

In [ ]:
def order_document_sections_by_query_similarity (query, contexts):
    """
    query in string format is given, context are basically document embeddings, we have calculated above, first this fn finds embeddings of string
    query then find similary docs
    """
    # query_embedding = get_embedding(query)
    query_embedding=get_embedding(query, engine=embedding_model)
    # document_similarities = sorted([
    #     (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    # ], reverse=True)
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ],reverse=True)
    return document_similarities

In [ ]:
doc_sim=order_document_sections_by_query_similarity("Policy documentation needs to be followed by each employee", test_doc_embeddings)

### 3) add the most relavant doc to query prompt

In [ ]:
MAX_SECTION_LEN = 500
SEPARATOR = "\n* "
ENCODING = "gpt2"  # encoding for text-davinci-003

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [ ]:
df_test = df_test.set_index(["title", "heading"])

In [ ]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.n_tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and in a concise manner, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"


In [ ]:
prompt = construct_prompt(
    "How many leaves I am entitled at 10Pearls?",
    test_doc_embeddings,
    df_test
)

print("===\n", prompt)

Selected 4 document sections:
('LEAVE ENTITLEMENT & RULE in 10Pearls Policies and Procedures', 'HAJJ/RELIGIOUS CONGREGATIONS')
('Definitions in 10Pearls Policies and Procedures', 'Long Leave')
('Definitions in 10Pearls Policies and Procedures', 'Short Leave')
('LEAVE ENTITLEMENT & RULE in 10Pearls Policies and Procedures', 'GENERAL')
===
 Answer the question as truthfully as possible using the provided context, and in a concise manner, and if the answer is not contained within the text below, say "I don't know."

Context:

* Long Leaves availed for religious purpose such as Hajj/Religious congregations can only be applied for once in two years and are subject to approval of HR and Delivery Directory.
* “Long Leave” shall mean leave of three or more days.
* “Short Leave” shall mean leave for a maximum of 2 hours in day.
* This policy has been framed bearing in mind the importance of ensuring the smooth running of the business of each department of the Company and equally important, the 

In [ ]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 300,
    "model": COMPLETIONS_MODEL,
}

In [ ]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings,
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

In [ ]:
answer_query_with_context("How many leaves I am entitled for", df_test, test_doc_embeddings)

Selected 1 document sections:
('LEAVE ENTITLEMENT & RULE in 10Pearls Policies and Procedures', 'SHORT AND LONG LEAVES')


'You are entitled to 4 short leaves and long leaves with approval from HR and Delivery Directors in a quarter.'

In [ ]:
answer_query_with_context("What is the leave policy at 10pearls?", df_test, test_doc_embeddings)

Selected 6 document sections:
('LEAVE ENTITLEMENT & RULE in 10Pearls Policies and Procedures', 'GENERAL')
('Definitions in 10Pearls Policies and Procedures', 'Long Leave')
('Definitions in 10Pearls Policies and Procedures', 'Short Leave')
('LEAVE ENTITLEMENT & RULE in 10Pearls Policies and Procedures', 'BEREAVEMENT')
('LEAVE ENTITLEMENT & RULE in 10Pearls Policies and Procedures', 'UNPAID LEAVES')
('LEAVE ENTITLEMENT & RULE in 10Pearls Policies and Procedures', 'HAJJ/RELIGIOUS CONGREGATIONS')


'Leave is a privilege only for Full-time Employees, and cannot be claimed as a right. Approval of leave is subject to the discretion of the Leave Sanctioning Authority. Long Leave shall mean leave of three or more days, and Short Leave shall mean leave for a maximum of 2 hours in day. In case of death of an immediate family member, Employees may avail 3 days of bereavement leave which shall not be deducted from the leave quota. Unpaid leaves will generally not be granted as excessive leaves may impact productivity. Long Leaves availed for religious purpose such as Hajj/Religious congregations can only be applied for once in two years and are subject to approval of HR and Delivery Directory.'

In [ ]:
answer_query_with_context("Tell me about leaves?", df_test, test_doc_embeddings)

Selected 2 document sections:
('LEAVE ENTITLEMENT & RULE in 10Pearls Policies and Procedures', 'SHORT AND LONG LEAVES')
('LEAVE ENTITLEMENT & RULE in 10Pearls Policies and Procedures', 'GENERAL')


'Employees can avail a maximum of 4 Short Leaves in a month and each short leave in a single day cannot exceed 2 hours. Long leaves must be applied and approved within COGS at the start of the quarter in which they are intended to be availed. Leave is subject to approval by the HR and Delivery Directors depending upon work situation. Leave is a privilege only for Full-time Employees, and cannot be claimed as a right.'

In [ ]:
answer_query_with_context("How many long leave can i apply", df_test, test_doc_embeddings)

Selected 2 document sections:
('LEAVE ENTITLEMENT & RULE in 10Pearls Policies and Procedures', 'SHORT AND LONG LEAVES')
('Definitions in 10Pearls Policies and Procedures', 'Long Leave')


'You can apply for long leaves with proper planning and spread them out throughout the year.'

In [ ]:
answer_query_with_context("I am working at 10pearls for last 1 years, am i eligible for loan?", df_test, test_doc_embeddings)

Selected 4 document sections:
('APPENDIX E LOAN POLICY in 10Pearls Policies and Procedures', 'POLICY OF LOAN AMOUNT')
('APPENDIX E LOAN POLICY in 10Pearls Policies and Procedures', 'GENERAL')
('APPENDIX E LOAN POLICY in 10Pearls Policies and Procedures', 'PAYBACK PERIOD')
('APPENDIX E LOAN POLICY in 10Pearls Policies and Procedures', 'OTHER RULES')


'No, you are not eligible for a loan as you need to have two years of tenure with 10pearls.'